In [1]:

import argparse
import math
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from sunpy.net import Fido
from sunpy.net import attrs as a
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
def fetch_hek_flares(start_time: str, end_time: str) -> pd.DataFrame:
    """Fetch HEK flare catalog via SunPy with helioprojective coordinates."""
    if Fido is None or a is None:
        raise ImportError("sunpy is required to fetch HEK flare catalog.")
    print(f"Fetching HEK flare catalog from {start_time} to {end_time}...")
    result = Fido.search(
        a.Time(start_time, end_time),
        a.hek.EventType("FL"),
    )
    hek_table = result["hek"]
    
    # Keep scalar columns plus coordinate columns we need
    scalar_cols = [name for name in hek_table.colnames if len(hek_table[name].shape) <= 1]
    coord_cols = [name for name in hek_table.colnames if any(coord_type in name for coord_type in 
                  ['hpc_coord', 'hgc_coord', 'hgs_coord', 'hpc_x', 'hpc_y'])]
    
    # Combine scalar and coordinate columns
    keep_cols = list(set(scalar_cols + coord_cols))
    
    dropped = set(hek_table.colnames) - set(keep_cols)
    if dropped:
        print(f"  Dropping HEK columns with multidimensional entries: {sorted(dropped)}")
    
    # Convert coordinate columns to string format before pandas conversion
    hek_subset = hek_table[keep_cols]
    for coord_col in coord_cols:
        if coord_col in hek_subset.colnames:
            # Convert coordinate objects to string representation
            coord_data = hek_subset[coord_col]
            hek_subset[coord_col] = [str(coord) for coord in coord_data]
    
    df = hek_subset.to_pandas()
    print(f"Fetched {len(df)} HEK flare entries")
    print(f"Available coordinate columns: {[col for col in df.columns if any(coord_type in col for coord_type in ['hpc', 'hgc', 'hgs'])]}")
    return df

/home/ggoodwin5/.conda/envs/aia-processor/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
fls = fetch_hek_flares("2023-05-01", "2023-08-15")

Fetching HEK flare catalog from 2023-05-01 to 2023-08-15...
Fetched 9665 HEK flare entries
Available coordinate columns: ['hgc_coord', 'hgs_coord', 'hpc_boundcc', 'hpc_x', 'hpc_heliosphere_corrected', 'hpc_bbox', 'hgs_y', 'hgc_boundcc', 'hpc_y', 'hpc_geom', 'hgs_boundcc', 'hgs_x', 'hpc_radius', 'hgc_y', 'hgc_x', 'hpc_coord', 'hgc_bbox', 'hgs_bbox']


In [4]:
fls

,fl_fluenceunit,kb_archivist,frm_name,boundbox_c2ur,fl_peakemunit,event_maskurl,refs_orig,active,area_atdiskcenter,event_peaktime,...,event_importance_num_ratings,event_c2error,obs_lastprocessingdate,hgs_bbox,obs_observatory,frm_versionnumber,event_coord2,event_clippedtemporal,intensunit,frm_specificid
0,,flare_detective,Flare Detective - Trigger Module,384.000000,,,,true,None,2023-05-01 00:00:08,...,,2,,"POLYGON((-9.554839 14.577554,-4.759562 14.5316...",SDO,0.51,345.600006,,,
1,,autosubmission_swpc,SWPC,89.900002,,,,true,None,2023-05-01 00:28:00,...,,90,,"POLYGON((-89.9 -89.9,89.9 -89.9,89.9 89.9,-89....",GOES,None,0.000000,,,
2,,freeland_samuel,SSW Latest Events,-23.000000,,,,true,None,2023-05-01 00:28:00,...,,0,,"POLYGON((1 1,1 1,1 1,1 1,1 1))",SDO,None,-24.000000,,,
3,,flare_detective,Flare Detective - Trigger Module,-307.200012,,,,true,None,2023-05-01 00:22:32,...,,2,,"POLYGON((39.176171 -27.006811,46.063378 -26.68...",SDO,0.51,-345.600006,,,
4,,flare_detective,Flare Detective - Trigger Module,-307.200012,,,,true,None,2023-05-01 00:22:22,...,,2,,"POLYGON((39.176156 -27.00675,46.063362 -26.680...",SDO,0.51,-345.600006,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9660,,flare_detective,Flare Detective - Trigger Module,230.399994,,,,true,None,2023-08-14 21:47:58,...,,2,,"POLYGON((-89.249976 8.718063,-81.481731 10.375...",SDO,0.51,192.000000,,,
9661,,flare_detective,Flare Detective - Trigger Module,153.600006,,,,true,None,2023-08-14 22:19:32,...,,2,,"POLYGON((-89.754745 4.399692,-63.951971 7.5814...",SDO,0.51,115.199997,,,
9662,,freeland_samuel,SSW Latest Events,9.000000,,,,true,None,2023-08-14 22:17:00,...,,0,,"POLYGON((1 1,1 1,1 1,1 1,1 1))",SDO,None,8.000000,,,
9663,,freeland_samuel,SSW Latest Events,9.000000,,,,true,None,2023-08-14 22:42:00,...,,0,,"POLYGON((1 1,1 1,1 1,1 1,1 1))",SDO,None,8.000000,,,


In [5]:
fls[fls['fl_goescls']!=''][['event_peaktime','obs_observatory','hpc_coord', 'fl_goescls']]

,event_peaktime,obs_observatory,hpc_coord,fl_goescls
1,2023-05-01 00:28:00,GOES,POINT(0 70.41699),C1.9
2,2023-05-01 00:28:00,SDO,POINT(572.46999 -338.961102),C1.9
5,2023-05-01 01:21:00,GOES,POINT(0 70.37385),M1.1
6,2023-05-01 01:21:00,SDO,POINT(583.843764 -339.71553),M1.1
7,2023-05-01 01:21:00,SDO,POINT(-906.20616 257.65365),M1.2
...,...,...,...,...
9654,2023-08-14 21:10:00,SDO,POINT(-928.92198 175.744566),C9.0
9655,2023-08-14 21:10:00,GOES,POINT(-911.76558 195.096624),C8.9
9662,2023-08-14 22:17:00,SDO,POINT(-917.88474 108.55179),C3.0
9663,2023-08-14 22:42:00,SDO,POINT(-917.88618 108.550308),C2.3
